In [1]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

name = "openai-community/gpt2" #"google/flan-t5-large"
path = f'./data/{name}/base_model'


hf = HuggingFacePipeline.from_model_id(
    model_id=name,
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 500},
model_kwargs={"cache_dir" : path , "temperature": 0, "max_length": 1000,}
    
)

hf.pipeline.model._validate_model_kwargs = lambda self: None


# hf = HuggingFacePipeline.from_model_id(
#     model_id= name,
#     task="text-generation",
#     pipeline_kwargs={"max_new_tokens": 10},
#     model_kwargs={"cache_dir" : path }
# )




/home/adil/repo/idriss/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "the capital of morroc is rabat give me a city of Morocco ?"

print(chain.invoke({"question": question}))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 The most commonly chosen electrode is the electrical current.


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.document_loaders import SeleniumURLLoader
# loader = PyPDFLoader("./cv.pdf")
# data = loader.load_and_split()

# loader1 = TextLoader("./data.txt")
# data1 = loader1.load_and_split()
# data.extend(data1)

urls = [
    "https://telquel.ma/instant-t/2023/07/28/le-ministere-de-lenseignement-superieur-lance-a-la-rentree-ses-ecoles-de-codages-code-212_1823860/",
    "https://leseco.ma/maroc/code-212-le-premier-centre-ouvrira-a-agadir.html",
]


loader = SeleniumURLLoader(urls=urls)

data = loader.load()

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [5]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(data, embeddings)

In [6]:
retriever = vectorstore.as_retriever()

In [7]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(hf, retriever=retriever)

In [8]:
user_input = "c est quoi code 212"

result = chain({"query": user_input})

response = result["result"]

print("Chatbot:", response)

/home/adil/repo/idriss/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Token indices sequence length is longer than the specified maximum sequence length for this model (4129 > 1024). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


IndexError: index out of range in self